In [4]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError

import requests
import subprocess
import lxml.html as lh
import html5lib

import time

from splinter import Browser

In [12]:
driver = subprocess.run(['which', 'chromedriver'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip()
executable_path = {'executable_path': driver}

def scraper(url):
    with Browser('chrome', **executable_path, headless=True) as browser:
        browser.visit(url)
        time.sleep(2)
        html = browser.html
    return html

In [21]:
scrape_dict = {}

# Nasa Website

In [ ]:
# 1. Nasa News Website
url1 = 'https://mars.nasa.gov/news/'
html1 = scraper(url1)

In [ ]:
# create the BS object
bs1 = BeautifulSoup(html1, 'html.parser')

In [ ]:
# Get the first title and it's description
scrape_dict['news_title'] = bs1.find('div', {'class': 'content_title'}).text
scrape_dict['news_description'] = bs1.find('div', {'class': 'article_teaser_body'}).text

# Image Search

In [ ]:
# Image Search
baseurl = 'https://www.jpl.nasa.gov'
url2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

html = scraper(url2)

In [28]:
bs2 = BeautifulSoup(html, 'html.parser')

In [29]:
image_route = bs2.find('a', {'class': 'button fancybox'})['data-fancybox-href']
scrape_dict['news_description'] = baseurl + image_route

# Mars Weather

In [ ]:
# 3. mars weather
url3 = 'https://twitter.com/marswxreport?lang=en'
html3 = scraper(url3)

In [14]:
bs3 = BeautifulSoup(html3, 'html.parser')

In [ ]:
scrape_dict['mars_weather'] = bs3.find('div', {'class': 'js-tweet-text-container'}).text.strip()

In [ ]:
# # Mars Facts
url = 'https://space-facts.com/mars/'
html = scraper(url)
bs = BeautifulSoup(html, 'lxml')
bs

In [12]:
mars_facts = str(bs.find('table', {'id': 'tablepress-mars'}))
mars_facts

'<table class="tablepress tablepress-id-mars" id="tablepress-mars">\n<tbody>\n<tr class="row-1 odd">\n<td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/>\n</td>\n</tr>\n<tr class="row-2 even">\n<td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/>\n</td>\n</tr>\n<tr class="row-3 odd">\n<td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.42 x 10^23 kg (10.7% Earth)</td>\n</tr>\n<tr class="row-4 even">\n<td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/deimos/">Deimos</a>)</td>\n</tr>\n<tr class="row-5 odd">\n<td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km (1.52 AU)</td>\n</tr>\n<tr class="row-6 even">\n<td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/>\n</td>\n</tr>\n<tr clas

In [15]:
mars_facts_df = pd.read_html(mars_facts)
mars_facts_df

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [ ]:
# # Mars Hemispheres
base_url = 'https://astrogeology.usgs.gov'
res = requests.get('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')
soup = BeautifulSoup(res.content, 'lxml')

In [25]:
# Get links to the hemisphere pages
items = soup.find_all('div', {'class': 'item'})
links = [base_url + item.a['href'] for  item in items]

<h3>Cerberus Hemisphere Enhanced</h3>

In [35]:
# open each of the websites and store the conent
hemisphere_image_urls = []
for link in links:
    temp_dict = {}
    res = requests.get(link)
    soup = BeautifulSoup(res.content, 'lxml')
    temp_dict['url'] = soup.find('a', {'target': '_blank'})['href']
    temp_dict['name'] = soup.find('h2', {'class': 'title'}).text.replace(' Enhanced', '')
    hemisphere_image_urls.append(temp_dict)

for d in hemisphere_image_urls:
    print(d)
    
    

{'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'name': 'Cerberus Hemisphere'}
{'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'name': 'Schiaparelli Hemisphere'}
{'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'name': 'Syrtis Major Hemisphere'}
{'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'name': 'Valles Marineris Hemisphere'}
